In [1]:
import pandas as pd
from keras.optimizers import SGD
from sklearn.metrics import classification_report
from tensorflow.python.keras.utils.np_utils import to_categorical
from package_from_scratch.CNNs.LeNet import LeNet
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

2023-05-01 12:36:37.003605: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-01 12:36:37.466892: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


ModuleNotFoundError: No module named 'package_from_scratch'

In [ ]:
# Load the training and test data
train_df = pd.read_csv('../data/digit-recognizer/train.csv')
test_df = pd.read_csv('../data/digit-recognizer/test.csv')

In [ ]:
# EDA
print(train_df.shape)
print(test_df.shape)

In [ ]:
# EDA
train_df.head()

In [ ]:
# Split the training data into features (X_train) and labels (y_train)
X_train = train_df.drop('label', axis=1)
y_train = train_df['label']
classes = train_df['label'].unique()
X_test = test_df

In [ ]:
# Normalize the data
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
# Reshape the data
X_train = X_train.values.reshape(-1,28,28,1)
X_test = X_test.values.reshape(-1,28,28,1)

In [ ]:
# One-hot encode the target variable
y_train = to_categorical(y_train, num_classes=10)

In [ ]:
# EDA
print(train_df.shape)
print(test_df.shape)

In [ ]:
# EDA
train_df.head()

In [ ]:
# Split the training data into features (X_train) and labels (y_train)
X_train = train_df.drop('label', axis=1)
y_train = train_df['label']
X_test = test_df

In [ ]:
# Normalize the data
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
# Reshape the data
X_train = X_train.values.reshape(-1,28,28,1)
X_test = X_test.values.reshape(-1,28,28,1)

In [ ]:
# One-hot encode the target variable
y_train = to_categorical(y_train, num_classes=10)

In [ ]:
# Split the train dataset into train and validation datasets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=2)

In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# initialize stochastic gradient descent with learning rate of 0.005
# how to tune learning rates ?????
opt = SGD(lr=0.005)
# opt = Adam(lr=0.001)
# Instantiate AlexNet architecture
# input image size 32x32
# output class is 3
LeNet_model = LeNet.build(width=28, height=28, depth=1,classes=len(classes))
LeNet_model.summary()
# compile the model
# loss function: cross-entropy and optimizer: SGD
LeNet_model.compile(loss="categorical_crossentropy",
                    optimizer=opt,
                    metrics=["accuracy"])
no_epochs = 1000
no_verbose = 1
no_batch_size = 32  
# 32 images will be presented to the network at a time,
# and a full forward and backward pass will be
# done to update the parameters of the network
# train the network

print("[INFO] training network...")

with tf.device('/gpu:1'):
    
    history = LeNet_model.fit(X_train, y_train,
                        validation_data=(X_val, y_val),
                        batch_size=no_batch_size,
                        epochs=no_epochs,
                        verbose=no_verbose)

    print("[INFO] evaluating network...")

predictions = LeNet_model.predict(X_test, batch_size=no_batch_size)

print(classification_report(
        y_val.argmax(axis=1),
        predictions.argmax(axis=1),
        target_names=classes
    ))


In [ ]:
# initialize stochastic gradient descent with learning rate of 0.005
# how to tune learning rates ?????
opt = SGD(lr=0.005)
# opt = Adam(lr=0.001)
# Instantiate AlexNet architecture
# input image size 32x32
# output class is 3
LeNet_model = LeNet.build(width=28, height=28, depth=1,classes=len(classes))
LeNet_model.summary()
# compile the model
# loss function: cross-entropy and optimizer: SGD
LeNet_model.compile(loss="categorical_crossentropy",
                    optimizer=opt,
                    metrics=["accuracy"])
no_epochs = 1000
no_verbose = 1
no_batch_size = 32  
# 32 images will be presented to the network at a time,
# and a full forward and backward pass will be
# done to update the parameters of the network
# train the network

print("[INFO] training network...")

with tf.device('/gpu:0'):
    
    history = LeNet_model.fit(X_train, y_train,
                        validation_data=(X_val, y_val),
                        batch_size=no_batch_size,
                        epochs=no_epochs,
                        verbose=no_verbose)

    print("[INFO] evaluating network...")

predictions = LeNet_model.predict(X_test, batch_size=no_batch_size)

print(classification_report(
        y_val.argmax(axis=1),
        predictions.argmax(axis=1),
        target_names=classes
    ))


In [ ]:
# Evaluate the model
val_loss, val_acc = LeNet_model.evaluate(X_val, y_val)
print('Validation Loss:', val_loss)
print('Validation Accuracy:', val_acc)

In [ ]:
# Plot the training and validation accuracy curves
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()

# Plot the training and validation loss curves
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Get the predictions on the test set
y_pred = LeNet_model.predict(X_test)

# Convert the predictions to integer labels
y_pred = np.argmax(y_pred, axis=1)

# Plot some examples of the test set with their predicted labels
num_examples = 9
plt.figure(figsize=(10,10))
for i in range(num_examples):
    plt.subplot(3, 3, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_test[i], cmap=plt.cm.binary)
    plt.xlabel('Predicted Label: {}'.format(y_pred[i]))
plt.show()

In [ ]:
# Save to csv file
submission = pd.DataFrame({'ImageId':list(range(1,len(predictions)+1)), 'Label':predictions})
submission.to_csv('submission.csv', index=False)

In [ ]:
# Save to csv file
submission = pd.DataFrame({'ImageId':list(range(1,len(predictions)+1)), 'Label':predictions})
submission.to_csv('submission.csv', index=False)

In [ ]:
# Get the predictions on the test set
y_pred = LeNet_model.predict(X_test)

# Convert the predictions to integer labels
y_pred = np.argmax(y_pred, axis=1)

# Plot some examples of the test set with their predicted labels
num_examples = 9
plt.figure(figsize=(10,10))
for i in range(num_examples):
    plt.subplot(3, 3, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_test[i], cmap=plt.cm.binary)
    plt.xlabel('Predicted Label: {}'.format(y_pred[i]))
plt.show()

In [ ]:
# Save to csv file
submission = pd.DataFrame({'ImageId':list(range(1,len(predictions)+1)), 'Label':predictions})
submission.to_csv('submission.csv', index=False)